In [5]:
%pip install -U langgraph langsmith langchain_anthropic langchain langchain-community langchainhub langchain-core langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain_ollama.llms import OllamaLLM

model = OllamaLLM(model="phi4")
model.invoke("Who is Thalapathy Vijay?")

'Thalapathy Vijay, whose real name is Sivakumar Pandian, is a prominent Indian actor known for his work in Tamil cinema. Born on January 16, 1977, in Vellore, Tamil Nadu, he began his career as a model before transitioning into acting. His breakthrough came with the movie "Padayappa" in 1999, directed by K.S. Ravikumar. However, it was his performances in films like "Udhaya," "Kuselan," and "Pudhupettai" that solidified his status as a leading actor.\n\nVijay\'s popularity soared with blockbusters such as "Sivaji: The Boss" (2007), "Thuppakki" (2012), and "Mersal" (2017). He is often referred to as "Thalapathy," which translates to "leader of soldiers," a title he earned after his role in the film "Pazhani." His films are known for their commercial appeal, combining elements of action, romance, and entertainment.\n\nVijay is not only celebrated for his acting skills but also for his philanthropic efforts. He has been actively involved in various charitable activities, including support

'<think>\nAlright, I need to figure out what "LangChain" refers to. It seems like it\'s a term related to AI and machine learning. I remember seeing some articles or tutorials about generative models before, maybe something called GANs? That\'s Generative Adversarial Networks. So perhaps LangChain is a tool that uses those concepts.\n\nI think the user provided a detailed answer earlier, breaking down what LangChain might be. It mentions it\'s an open-source AI framework for machine learning and NLP tasks like text generation, translation, summarization, etc. That makes sense because generative models are widely used in these areas.\n\nThe article also talked about how LangChain integrates with popular frameworks like PyTorch, TensorFlow, and JAX. This integration would allow developers to use advanced models without writing custom code. The flexibility is probably one of the key features people look for when choosing AI tools.\n\nAnother point was that LangChain supports both text gen